In [8]:
import pandas as pd
import numpy as np
import requests
import os
from geopy.geocoders import Nominatim 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
import seaborn as sns
import folium

print('Complete')

Complete


In [9]:
CLIENT_ID = '1TBZHYVQKGUQUK3GVHMQF0WL2AA2ZIVL1GR5Q0EYTSQJ1FQG' 
CLIENT_SECRET = 'R2PMIA1ZWVCBDYHLAT23TR4Z3ULX0SS4CADIOEAUCGAIFJBL' 
VERSION = '20201226'

In [10]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude


def get_venues(lat,lng):
    #set variables
    radius=500
    LIMIT=100
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


def get_venue_details(venue_id):
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    # get all the data
    results = requests.get(url).json()
    print(results)
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return new_york_data

In [11]:
new_york_data=get_new_york_data()
new_york_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [13]:
column_names=['Borough', 'Neighborhood', 'ID','Name']
veg_rest_ny=pd.DataFrame(columns=column_names)
count=1
for row in new_york_data.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    veg_restaurants=venues[venues['Category']=='Vegetarian / Vegan Restaurant']   
    print('(',count,'/',len(new_york_data),')','Vegetarian/Vegan Resturants in '+Neighborhood+', '+Borough+':'+str(len(veg_restaurants)))
    for restaurant_detail in veg_restaurants.values.tolist():
        id, name , category=restaurant_detail
        veg_rest_ny = veg_rest_ny.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1

KeyError: 'groups'

There are 306 restaurants analyzed, so expect the runtime to take 5-min

In [ ]:
veg_rest_ny.to_csv('veg_rest_tocsv1.csv')
veg_rest_ny = pd.read_csv('veg_rest_tocsv1.csv',index_col=0)
veg_rest_ny.tail()

In [ ]:
veg_rest_ny.shape

In [ ]:
NumNeighbor = 10 
veg_rest_ny.groupby('Neighborhood')['ID'].count().nlargest(NumNeighbor).plot.bar(figsize=(20,4))
plt.title('Vegetarian/Vegan Resturants per Neighborhood: New York', fontsize = 20)
plt.xlabel('Neighborhood', fontsize = 15)
plt.ylabel('Vegetarian/Vegan Resturants', fontsize=15)
plt.xticks(rotation = 'horizontal')
plt.show()

In [ ]:
column_names=['Borough', 'Neighborhood', 'ID','Name','Likes','Rating','Price']
veg_rest_ny_csv=pd.DataFrame(columns=column_names)
count=1
for row in veg_rest_ny.values.tolist():
    Borough,Neighborhood,ID,Name=row
    try:
        venue_details=get_venue_details(ID)
        print(venue_details)
        id,name,likes,rating,price=venue_details.values.tolist()[0]
    except IndexError:
        print('No data available for id=',ID)
        id,name,likes,rating,price=[0]*5
    print('(',count,'/',len(veg_rest_ny),')','processed')
    veg_rest_ny_csv = veg_rest_ny_csv.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Price' : price
                                               }, ignore_index=True)
    count+=1

In [ ]:
veg_rest_ny_csv.head()

In [ ]:
veg_rest_ny_csv.to_csv('veg_rest_ny.csv')

In [ ]:
veg_rest_ny_csv.shape

In [ ]:
veg_rest_ny_csv['Likes'] = veg_rest_ny_csv['Likes'].astype('float64')
veg_rest_ny_csv['Rating'] = veg_rest_ny_csv['Rating'].astype('float64')
veg_rest_ny_csv['Price'] = veg_rest_ny_csv['Price'].astype('float64')
veg_rest_ny_csv.info()

In [ ]:
veg_rest_ny_csv.iloc[veg_rest_ny_csv['Likes'].idxmax()]

In [ ]:
veg_rest_ny_csv.iloc[veg_rest_ny_csv['Price'].idxmax()]

In [ ]:
veg_rest_ny_csv.iloc[veg_rest_ny_csv['Rating'].idxmax()]

In [ ]:
ny_neighborhood_stats=veg_rest_ny_csv.groupby('Name',as_index=False).mean()[['Name','Price']]
ny_neighborhood_stats.columns=['Name','Price']
ny_neighborhood_stats.sort_values(['Price'],ascending=False).head(10)

The prices seem a bit off for the top values, so I will change the values to make the prices value by the number of $ signs, from 1-4. This is not such a big dataset, because there are 42 rows.

In [ ]:
veg_rest_ny_csv['Price'] = veg_rest_ny_csv['Price'].replace([273],2)
veg_rest_ny_csv['Price'] = veg_rest_ny_csv['Price'].replace([218],2)
veg_rest_ny_csv['Price'] = veg_rest_ny_csv['Price'].replace([156],3)
veg_rest_ny_csv.loc[14,'Price'] = 2
veg_rest_ny_csv.loc[15,'Price'] = 2
veg_rest_ny_csv.loc[19,'Price'] = 2
veg_rest_ny_csv.loc[21,'Price'] = 2
veg_rest_ny_csv.loc[41,'Price'] = 2
veg_rest_ny_csv.loc[20,'Price'] = 2
veg_rest_ny_csv.loc[18,'Price'] = 1
veg_rest_ny_csv.loc[22,'Price'] = 2
veg_rest_ny_csv.loc[27,'Price'] = 2
veg_rest_ny_csv.loc[39,'Price'] = 2
veg_rest_ny_csv.loc[1,'Price'] = 2
veg_rest_ny_csv.loc[2,'Price'] = 1
veg_rest_ny_csv.loc[28,'Price'] = 2
veg_rest_ny_csv.loc[17,'Price'] = 2
veg_rest_ny_csv.loc[9,'Price'] = 2
veg_rest_ny_csv.loc[29,'Price'] = 1
veg_rest_ny_csv.loc[25,'Price'] = 2
veg_rest_ny_csv.loc[26,'Price'] = 2
veg_rest_ny_csv.loc[13,'Price'] = 2
veg_rest_ny_csv.loc[24,'Price'] = 2
veg_rest_ny_csv.loc[23,'Price'] = 2
veg_rest_ny_csv.loc[10,'Price'] = 2
veg_rest_ny_csv.loc[36,'Price'] = 2
veg_rest_ny_csv.loc[35,'Price'] = 2
veg_rest_ny_csv.loc[32,'Price'] = 2
veg_rest_ny_csv.loc[34,'Price'] = 1
veg_rest_ny_csv.loc[11,'Price'] = 2
veg_rest_ny_csv.loc[12,'Price'] = 2
veg_rest_ny_csv.loc[33,'Price'] = 2
veg_rest_ny_csv.loc[40,'Price'] = 2
veg_rest_ny_csv.loc[3,'Price'] = 2
veg_rest_ny_csv.loc[31,'Price'] = 2
veg_rest_ny_csv.loc[37,'Price'] = 3
veg_rest_ny_csv.loc[5,'Price'] = 2
veg_rest_ny_csv.loc[6,'Price'] = 2
veg_rest_ny_csv.loc[7,'Price'] = 2

In [ ]:
veg_rest_ny_csv.sort_values(['Price'],ascending=False).head(10)

In [ ]:
veg_rest_ny_csv.sort_values(['Price'],ascending=True).head(10)

There are 3 blank values that need to be dropped.

In [ ]:
veg_rest_ny_csv = veg_rest_ny_csv.drop([0, 38, 30])

In [ ]:
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(veg_rest_ny_csv, title = "Download CSV file", filename = "veg_restaurant_data.csv"):  
    csv = veg_rest_ny_csv.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(veg_rest_ny_csv)


In [ ]:
veg_rest_ny_csv.sort_values(['Price'],ascending=True).head(10)

In [ ]:
veg_rest_ny_csv.sort_values(['Rating'],ascending=False).head(10)

In [ ]:
veg_rest_ny_csv.sort_values(['Rating'],ascending=True).head(10)

In [ ]:
veg_rest_ny_csv[veg_rest_ny_csv['Price'] == 1].count()

In [ ]:
veg_rest_ny_csv[veg_rest_ny_csv['Price'] == 2].count()

In [ ]:
veg_rest_ny_csv[veg_rest_ny_csv['Price'] == 3].count()

Which neighborhoods have the best rated vegetarian/vegan restaurants?

In [ ]:
veg_rest_ny_csv.groupby('Neighborhood').mean()['Rating'].plot.bar(figsize=(10,5))
plt.title('Average rating of Vegtarian/Vegan Restaurants in each neighborhood', fontsize=20)
#On x-axis
plt.xlabel('Neighborhood', fontsize = 15)
#On y-axis
plt.ylabel('Average Rating', fontsize=15)
plt.xticks(rotation = 'vertical')
plt.show()